In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import json 
from scipy.sparse import load_npz

In [2]:
# take the labels from the y matrix, then put them into T1 format in the aux tasks

In [3]:
T2 = pd.read_csv('../../ci8b00670_si_002/Label_Matrix/T_files/all/T2.csv')

In [4]:
T2_hofmarcher = pd.read_csv('../../ci8b00670_si_002/Label_Matrix/T_files/images_Hofmarcher/T2.csv')

In [5]:
T2

,input_compound_id,smiles
0,1075529,Cc1cc(NC(=O)Nc2ccc(cc2)C(=O)O)c3ccccc3n1
1,1203610,Clc1ccc(cc1)C(N2CCN(CC2)C(=O)Nc3ccccc3)c4cccnc4
2,1203845,CS(=O)(=O)N1CCN(CC1)C(c2ccc(Cl)cc2)c3cccnc3
3,1203966,Fc1cc(Cl)ccc1C(N2CCN(CC2)C(=O)C3CCCCC3)c4cccnc4
4,2176418,[O-][N+](=O)c1ccccc1S(=O)(=O)Nc2ccc(cc2)c3oc4c...
...,...,...
520391,2275345,[O-][N+](=O)c1cccc(c1)-c1nc2ccccc2c(=O)o1
520392,2275362,[O-][N+](=O)c1ncccc1OCC(=O)Nc1ccc2CCCc2c1
520393,2275364,[O-][n+]1onc-2c1CCc1nnccc-21
520394,2275365,c1ccc(cc1)-c1cnc(nn1)-c1ccccn1


In [6]:
T2_hofmarcher

,input_compound_id,smiles
0,845253,BrC1=CN2CCS(=O)(=O)N=C2C(Br)=C1
1,298290,Brc1c(Br)c(Br)c2[nH]nnc2c1Br
2,922657,Brc1c(CSc2nc3ccccc3s2)nc2ncccn12
3,62817,Brc1c(NC2=NCCN2)ccc2nccnc12
4,1001579,Brc1c(cnn(-c2ccccc2)c1=O)N1CCCCC1
...,...,...
10570,773366,c1csc(c1)-c1nc(ncc1-c1nnnn1-c1ccccc1)-c1ccncc1
10571,989091,c1csc(c1)-c1ncc2ccccn12
10572,26590,c1nc(cs1)-c1nc2ccccc2[nH]1
10573,736500,c1nn(-c2ccccc2)c2ncn3cnnc3c12


In [8]:
path = '../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/results/T10c_cont.csv'
t10c_baseline = pd.read_csv(path)

In [9]:
t10c_baseline.shape

(3681342, 12)

In [10]:
path = '../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/matrices/cls/cls_T10_y.npz'
t10_y_baseline = load_npz(path)

In [11]:
t10_y_baseline.shape

(419424, 2950)

In [13]:
# cont_classification_task_id corresponds to the shape of the Y matrix 

In [12]:
path = './files/T0_image_pseudolabels_full.csv'
t0_image = pd.read_csv(path)

In [13]:
t0_image

,melloddy_col_idx,input_assay_id,source,assay_type,assay_type_reason,model_description,assay_description
0,548,10000000,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_11,img_11
1,549,10000001,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_12,img_12
2,558,10000002,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_14,img_14
3,561,10000003,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_16,img_16
4,562,10000004,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_17,img_17
...,...,...,...,...,...,...,...
56,2941,10000056,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_276,img_276
57,2942,10000057,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_277,img_277
58,2943,10000058,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_278,img_278
59,2944,10000059,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_279,img_279


In [14]:
melloddy_col_idxs = t0_image['melloddy_col_idx']

In [15]:
melloddy_col_idxs

0      548
1      549
2      558
3      561
4      562
      ... 
56    2941
57    2942
58    2943
59    2944
60    2945
Name: melloddy_col_idx, Length: 61, dtype: int64

In [16]:
t10c.head()

,classification_task_id,descriptor_vector_id,fold_id,input_assay_id,standard_qualifier,standard_value,threshold,class_label,is_active,is_inactive,cont_classification_task_id,cont_descriptor_vector_id
0,2.0,44942,1,517,=,7.51,7.0,1.0,True,False,0,37563
1,2.0,47041,1,517,=,6.35,7.0,-1.0,False,True,0,39160
2,2.0,125427,1,517,=,6.87,7.0,-1.0,False,True,0,104582
3,2.0,283377,0,517,=,8.70,7.0,1.0,True,False,0,235792
4,2.0,47027,3,517,=,8.40,7.0,1.0,True,False,0,39149


In [17]:
t10c_lim = t10c_baseline.query('cont_classification_task_id in @melloddy_col_idxs')

In [18]:
t10c_lim

,classification_task_id,descriptor_vector_id,fold_id,input_assay_id,standard_qualifier,standard_value,threshold,class_label,is_active,is_inactive,cont_classification_task_id,cont_descriptor_vector_id
159803,1071.0,35779,2,688141,=,4.30,4.2,1.0,True,False,548,29941
159804,1071.0,194820,1,688141,=,4.80,4.2,1.0,True,False,548,162422
159805,1071.0,429335,0,688141,=,5.05,4.2,1.0,True,False,548,357522
159806,1071.0,232855,2,688141,=,4.50,4.2,1.0,True,False,548,193669
159807,1071.0,461174,2,688141,=,4.80,4.2,1.0,True,False,548,384914
...,...,...,...,...,...,...,...,...,...,...,...,...
3680946,4892.0,293233,2,4000027,=,-1.00,0.0,-1.0,False,True,2945,243832
3680947,4892.0,82966,2,4000027,=,-1.00,0.0,-1.0,False,True,2945,69371
3680948,4892.0,39158,0,4000027,=,-1.00,0.0,-1.0,False,True,2945,32940
3680949,4892.0,399299,1,4000027,=,1.00,0.0,1.0,True,False,2945,332453


In [21]:
with open('./files/image_iai_cont_map.json') as fp:
    image_iai_cont_map = json.load(fp)
with open('./files/image_iai_cont_map_2.json') as fp:
    image_iai_cont_map_2 = json.load(fp)

In [22]:
image_iai_cont_map = {int(k):int(v) for k,v in image_iai_cont_map.items()}
image_iai_cont_map_2 = {int(k):int(v) for k,v in image_iai_cont_map_2.items()}



In [23]:
map_dict = {image_iai_cont_map_2[i] : image_iai_cont_map[i] for i in image_iai_cont_map}

In [25]:
t10c_lim['input_assay_id'] = t10c_lim['cont_classification_task_id'].map(map_dict)

/home/rama.jabal/.conda/envs/melloddy_tuner_regression/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
t10c_lim['standard_value'] = np.where(t10c_lim['class_label'] == -1 , 0, 6)

/home/rama.jabal/.conda/envs/melloddy_tuner_regression/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
t10c_lim['standard_qualifier'] = '='

/home/rama.jabal/.conda/envs/melloddy_tuner_regression/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
t10c_lim

,classification_task_id,descriptor_vector_id,fold_id,input_assay_id,standard_qualifier,standard_value,threshold,class_label,is_active,is_inactive,cont_classification_task_id,cont_descriptor_vector_id
159803,1071.0,35779,2,10000000,=,6,4.2,1.0,True,False,548,29941
159804,1071.0,194820,1,10000000,=,6,4.2,1.0,True,False,548,162422
159805,1071.0,429335,0,10000000,=,6,4.2,1.0,True,False,548,357522
159806,1071.0,232855,2,10000000,=,6,4.2,1.0,True,False,548,193669
159807,1071.0,461174,2,10000000,=,6,4.2,1.0,True,False,548,384914
...,...,...,...,...,...,...,...,...,...,...,...,...
3680946,4892.0,293233,2,10000060,=,0,0.0,-1.0,False,True,2945,243832
3680947,4892.0,82966,2,10000060,=,0,0.0,-1.0,False,True,2945,69371
3680948,4892.0,39158,0,10000060,=,0,0.0,-1.0,False,True,2945,32940
3680949,4892.0,399299,1,10000060,=,6,0.0,1.0,True,False,2945,332453


In [29]:
t10c_lim.shape

(1307684, 12)

In [31]:
path = '../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/mapping_table/T5.csv'
t5_baseline = pd.read_csv(path)

In [32]:
t5_baseline

,input_compound_id,fold_id,descriptor_vector_id
0,1075529,3,405745
1,1203610,2,410876
2,1203845,2,464910
3,1203966,2,7212
4,2176418,0,361020
...,...,...,...
520218,2275345,4,172227
520219,2275362,0,394748
520220,2275364,3,483801
520221,2275365,2,74868


In [33]:
df_mgd = pd.merge(
    t5_baseline[['input_compound_id','descriptor_vector_id']].drop_duplicates()
    ,t10c_lim
    ,on='descriptor_vector_id'
    ,how='inner'
)

In [45]:
# same desc vector id can be linked to several input cmpd ids 

In [34]:
df_mgd.shape

(1364059, 13)

In [35]:
df_mgd.head()

,input_compound_id,descriptor_vector_id,classification_task_id,fold_id,input_assay_id,standard_qualifier,standard_value,threshold,class_label,is_active,is_inactive,cont_classification_task_id,cont_descriptor_vector_id
0,9495,218506,1071.0,4,10000000,=,6,4.2,1.0,True,False,548,181369
1,9495,218506,1072.0,4,10000001,=,0,4.7,-1.0,False,True,549,181369
2,9495,218506,1093.0,4,10000005,=,0,4.7,-1.0,False,True,569,181369
3,9495,218506,1235.0,4,10000012,=,0,5.0,-1.0,False,True,680,181369
4,9495,218506,1241.0,4,10000014,=,0,5.0,-1.0,False,True,686,181369


In [36]:
df_t1_truelabels_out = df_mgd[['input_assay_id','standard_value','standard_qualifier','input_compound_id']].drop_duplicates()

In [37]:
df_t1_truelabels_out.shape

(1364059, 4)

In [50]:
df_t1_truelabels_out.to_csv('./files/T1_image_truelabels.csv', index=False)

In [78]:
df_t1_truelabels_out.input_compound_id.unique().shape

(274091,)